In [1]:
import json
import hashlib
from datetime import datetime
from breeze_connect import BreezeConnect
import urllib
import pandas as pd
import numpy as np
import math
from trading_api.icici import icici
import datetime as dt

In [1]:
import logging

# Create a logger object
log = logging.getLogger()
log.setLevel(logging.DEBUG)  # Set the root logger level to DEBUG


# Create a console handler for printing logs to console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)  # Set console handler level to INFO

# Define format for both handlers
formatter = logging.Formatter('[%(asctime)s][%(levelname)s] - %(message)s')
console_handler.setFormatter(formatter)

# Add the handlers to the logger

log.addHandler(console_handler)



In [3]:
api_session='37289997'

In [3]:
icici1 = icici(api_session=api_session)
icici1.login()

Exception: Could not authenticate credentials. Please check token and keys

In [2]:
stock = 'ITC'
from_date = '2024-03-19'
to_date = '2024-03-20'
interval = '1minute'
exchange = 'NSE'

df = icici1.get_histroy_data_stock(stock,from_date,to_date,interval=interval,exchange=exchange)


NameError: name 'icici1' is not defined

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    398 non-null    object 
 1   stock_code  398 non-null    object 
 2   open        398 non-null    float64
 3   close       398 non-null    float64
 4   high        398 non-null    float64
 5   low         398 non-null    float64
dtypes: float64(4), object(2)
memory usage: 18.8+ KB


In [87]:
df['date_time'] = pd.to_datetime(df['datetime'])

In [88]:
df

,datetime,stock_code,open,close,high,low,date_time
0,2024-03-19 09:08:00,ITC,418.00,418.00,418.00,418.00,2024-03-19 09:08:00
1,2024-03-19 09:15:00,ITC,418.15,418.40,418.40,418.00,2024-03-19 09:15:00
2,2024-03-19 09:16:00,ITC,418.40,418.15,418.95,418.00,2024-03-19 09:16:00
3,2024-03-19 09:17:00,ITC,418.35,418.65,418.75,418.10,2024-03-19 09:17:00
4,2024-03-19 09:18:00,ITC,418.65,419.00,419.30,418.45,2024-03-19 09:18:00
...,...,...,...,...,...,...,...
393,2024-03-19 15:56:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:56:00
394,2024-03-19 15:57:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:57:00
395,2024-03-19 15:58:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:58:00
396,2024-03-19 15:59:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:59:00


In [89]:
df.drop(df.index[(df.date_time.dt.hour> 14) & (df.date_time.dt.minute >19)],inplace=True)

In [90]:
df['change'] = df['close']-df['open']

In [91]:
df['per_change'] = (df['change']/df['open'])*100

In [92]:
df['per_change'].mean()

-0.006618667089363048

In [94]:
df['change'].mean()

-0.02752043596730205

In [95]:
for index,row in df.iterrows():
    trade = pd.DataFrame()
    print(row['datetime'])

2024-03-19 09:08:00
2024-03-19 09:15:00
2024-03-19 09:16:00
2024-03-19 09:17:00
2024-03-19 09:18:00
2024-03-19 09:19:00
2024-03-19 09:20:00
2024-03-19 09:21:00
2024-03-19 09:22:00
2024-03-19 09:23:00
2024-03-19 09:24:00
2024-03-19 09:25:00
2024-03-19 09:26:00
2024-03-19 09:27:00
2024-03-19 09:28:00
2024-03-19 09:29:00
2024-03-19 09:30:00
2024-03-19 09:31:00
2024-03-19 09:32:00
2024-03-19 09:33:00
2024-03-19 09:34:00
2024-03-19 09:35:00
2024-03-19 09:36:00
2024-03-19 09:37:00
2024-03-19 09:38:00
2024-03-19 09:39:00
2024-03-19 09:40:00
2024-03-19 09:41:00
2024-03-19 09:42:00
2024-03-19 09:43:00
2024-03-19 09:44:00
2024-03-19 09:45:00
2024-03-19 09:46:00
2024-03-19 09:47:00
2024-03-19 09:48:00
2024-03-19 09:49:00
2024-03-19 09:50:00
2024-03-19 09:51:00
2024-03-19 09:52:00
2024-03-19 09:53:00
2024-03-19 09:54:00
2024-03-19 09:55:00
2024-03-19 09:56:00
2024-03-19 09:57:00
2024-03-19 09:58:00
2024-03-19 09:59:00
2024-03-19 10:00:00
2024-03-19 10:01:00
2024-03-19 10:02:00
2024-03-19 10:03:00


In [101]:
nd = pd.DataFrame(columns=df.columns)

In [103]:
nd = nd.append(df.sample)

AttributeError: 'DataFrame' object has no attribute 'append'

In [45]:
import yfinance as yf

# Ticker symbol for ITC
ticker_symbol = "^NSEBANK"  # For the National Stock Exchange of India

# Fetch historical data
data = yf.download(ticker_symbol, period="6d", interval="1m")

print(data)


[*********************100%%**********************]  1 of 1 completed

                                   Open          High           Low  \
Datetime                                                              
2024-03-13 09:15:00+05:30  47341.148438  47436.949219  47340.250000   
2024-03-13 09:16:00+05:30  47430.851562  47430.851562  47374.050781   
2024-03-13 09:17:00+05:30  47379.148438  47418.500000  47378.398438   
2024-03-13 09:18:00+05:30  47411.199219  47412.750000  47367.148438   
2024-03-13 09:19:00+05:30  47396.250000  47435.101562  47387.101562   
...                                 ...           ...           ...   
2024-03-20 15:25:00+05:30  46338.449219  46349.398438  46331.550781   
2024-03-20 15:26:00+05:30  46347.550781  46353.750000  46309.601562   
2024-03-20 15:27:00+05:30  46317.800781  46346.351562  46314.101562   
2024-03-20 15:28:00+05:30  46344.750000  46378.449219  46339.699219   
2024-03-20 15:29:00+05:30  46370.750000  46386.300781  46365.851562   

                                  Close     Adj Close  Volume  
Datetime    

In [46]:
data['Change']=data['Close']-data['Open']

In [47]:
data['Pre_Change']= (data['Change']/data['Open'])*100

In [48]:
old_price_long = 0
old_price_short = 0
target_prencantage = 0.1
stop_loss_precentage = 0.2 
stop_loss_long = False
stop_loss_short = False
stop_loss_price_long = 0
stop_loss_price_short = 0
long_hold = False
short_hold = False
trade_book = []
qty = 30
brokerage = 20
stt = 0.025/100
transaction_charge = 0.000325/100
stamp = 0.003/100
sebi_charges = 10

for index, row in data.iterrows():

    
    current_price = row['Close']
    print(index,current_price)
    pre_incrase = ((current_price-old_price_long)/old_price_long)*100 if old_price_long != 0 else 0
    
    if (pre_incrase >= target_prencantage) and long_hold :
        long_hold = False
        profit_long = current_price-old_price_long
        print(f'selling at a price {current_price}. profit = {(current_price-old_price_long)*qty} \n')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':charges,'profit_after_charges':(profit_long*qty)-charges})

    if pre_incrase<=-stop_loss_precentage and long_hold:
        long_hold = False
        stop_loss_long = True
        loss = old_price_long-current_price
        print('Stop Loss Triggered on long')
        print(f'Booked a Loss of {-loss} at price {current_price}')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':-loss*qty,'charges':charges,'profit_after_charges':(-loss*qty)-charges})

    if (not long_hold) and (not stop_loss_long):
        print('Got False going long')
        long_hold = True
        print(f'Buying at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_long = row['Close']
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stamp_rs = order_value*stamp
        charges = brokerage+stamp_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'long','price':current_price,'order_value':int(order_value),'profit':0,'charges':charges,'profit_after_charges':-charges})


    pre_dec = ((-current_price+old_price_short)/old_price_short)*100 if old_price_short != 0 else 0
    if (pre_dec >= target_prencantage) and short_hold:
        short_hold = False
        profit_short = -current_price+old_price_short
        print(f'selling at a price {current_price}. profit = {(-current_price+old_price_short)*qty} \n')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stamp_rs = order_value*stamp
        charges = brokerage+stamp_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':charges,'profit_after_charges':(profit_short*qty)-charges})
    
    if pre_dec<=-stop_loss_precentage and short_hold:
        short_hold = False
        stop_loss_short = True
        loss = +old_price_short-current_price
        print('Stop Loss Triggered on short')
        print(f'Booked a Loss of {-loss} at price {current_price}')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':-loss*qty,'charges':charges,'profit_after_charges':(-loss*qty)-charges})

    
    if (not short_hold) and (not stop_loss_short):
        
        print('Got False going short')
        short_hold = True
        print(f'Buying at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_short = row['Close']
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'short','price':current_price,'order_value':int(order_value),'profit':0,'charges':charges,'profit_after_charges':0-charges})
    
    

    

    if index.hour==15 and index.minute==29:
        if long_hold:
            long_hold = False
            profit_long = current_price-old_price_long
            print(f'Ending Trade for day {current_price}. profit = {(current_price-old_price_long)*qty} \n')
            order_value = qty*current_price
            tc = order_value*transaction_charge
            stt_rs = order_value*stt
            charges = brokerage+stt_rs+brokerage+sebi_charges
            trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':charges,'profit_after_charges':(profit_long*qty)-charges})


        if short_hold:
            short_hold = False
            profit_short = -current_price+old_price_short
            print(f'Ending Short at a price {current_price}. profit = {(-current_price+old_price_short)*qty} \n')
            order_value = qty*current_price
            tc = order_value*transaction_charge
            stamp_rs = order_value*stamp
            charges = brokerage+stamp_rs+brokerage+sebi_charges
            trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':charges,'profit_after_charges':(profit_short*qty)-charges})
            
    
        break




    
    


2024-03-13 09:15:00+05:30 47430.3515625
Got False going long
Buying at a price 47430.3515625. total buy price 1422910.546875 
 
Got False going short
Buying at a price 47430.3515625. total buy price 1422910.546875 
 
2024-03-13 09:16:00+05:30 47376.25
selling at a price 47376.25. profit = 1623.046875 

Got False going short
Buying at a price 47376.25. total buy price 1421287.5 
 
2024-03-13 09:17:00+05:30 47409.6484375
2024-03-13 09:18:00+05:30 47408.8515625
2024-03-13 09:19:00+05:30 47422.8984375
2024-03-13 09:20:00+05:30 47370.25
2024-03-13 09:21:00+05:30 47381.19921875
2024-03-13 09:22:00+05:30 47403.44921875
2024-03-13 09:23:00+05:30 47409.1484375
2024-03-13 09:24:00+05:30 47389.69921875
2024-03-13 09:25:00+05:30 47370.3515625
2024-03-13 09:26:00+05:30 47364.0
2024-03-13 09:27:00+05:30 47344.6484375
2024-03-13 09:28:00+05:30 47371.80078125
2024-03-13 09:29:00+05:30 47338.94921875
2024-03-13 09:30:00+05:30 47336.69921875
2024-03-13 09:31:00+05:30 47325.44921875
Stop Loss Triggered o

In [ ]:
old_price_long = 0
old_price_short = 0
target_prencentage = 0.1
stop_loss_precentage = 0.2 
stop_loss_long = False
stop_loss_short = False
stop_loss_price_long = 0
stop_loss_price_short = 0
long_hold = False
short_hold = False
trade_book = []
qty = 30
brokerage = 20
stt = 0.025/100
transaction_charge = 0.000325/100
stamp = 0.003/100
sebi_charges = 10

for index, row in data.iterrows():

    log.debug(f"variables values when running on_ticks getting ticks :- old_price_long : {old_price_long},old_price_short : {old_price_short},target_prencantage : {target_prencentage},stop_loss_precentage : {stop_loss_precentage},long_hold : {long_hold},short_hold : {short_hold},trade_book : {trade_book},qty : {qty}")
    #records_ticks(ticks['datetime'],ticks['open'],ticks['high'],ticks['low'],ticks['close'])

    

    current_price = row['Close']
    date = row['Datetime']

    per_change_long = ((current_price-old_price_long)/old_price_long)*100 if old_price_long != 0 else 0
    per_change_short = ((current_price-old_price_short)/old_price_short)*100 if old_price_short != 0 else 0
    price_change_long = current_price-old_price_long
    price_change_short = current_price-old_price_short
    log.info(f"% Change in Price for Long position = {per_change_long}")
    log.info(f"% Change in Price for Short position = {per_change_short} NOTE :: Negetive means profit in short trade same formula has been used to both long and short") 
    
    if (per_change_long >= target_prencentage) and long_hold :

        log.info(f"Exiting Long Trade since Both the Condition Satisfied per_change_long = {per_change_long} is greater than target_precentage = {target_prencentage} and Holding Position for Long is = {long_hold}")
        long_hold = False
        log.debug(f"setting long_hold to False.value after setting false = {long_hold}")
        profit_long = price_change_long
        log.info(f"Sold at {current_price} * {qty} = {current_price*qty}. Booked Profit of {profit_long} * {qty} = {profit_long*qty}\n")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty})
        log.debug(f"Appended to Trade book ('date':{date},'type':'exit long','price':{current_price},'order_value':{int(order_value)},'profit':{profit_long*qty})")

    if per_change_long <= -stop_loss_precentage and long_hold:

        log.warning(f"Stoploss on Long Position Triggered since Both Condition per_change_long = {per_change_long} is greater than Stoploss = {stop_loss_precentage} in absolute values and Long_hold is True = {long_hold}. NOTE per_change_long should be negetive since it is decreasing.")
        long_hold = False
        log.debug(f"Assigned False value to long_hold : {long_hold}")
        stop_loss_long = True
        log.debug(f"Changed stop_loss_long value to True since it is triggered. new value = {stop_loss_long}")
        loss = current_price-old_price_long
        log.warning(f"Exiting Long Position Booked a LOSS of {loss} * {qty} = {loss*qty}\n")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit long STOPLOSS','price':current_price,'order_value':int(order_value),'profit':loss*qty})
        log.debug(f"Appended to Trade book ('date':{date},'type':'exit long STOPLOSS','price':{current_price},'order_value':{int(order_value)},'profit':{loss*qty})")

    if (not long_hold) and (not stop_loss_long):

        log.info(f"Going Long Since not Holding and neither stoploss is triggered values :: long_hold : {long_hold}, stop_loss_long : {stop_loss_long}")
        long_hold = True

        log.info(f'Buying at a price {current_price} * {qty}. total buy price {qty*current_price} \n ')
        old_price_long = current_price
        log.debug(f"Setting global old_price_long to {current_price} and long_hold to {long_hold}")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'long','price':current_price,'order_value':int(order_value),'profit':0})


    
    if (per_change_short <= -target_prencentage) and short_hold:
        log.debug(f"Exiting Short Trade since Both Condition Satisfied. per_change_short : {per_change_short} is greater than target_prencentage : {target_prencentage} and short hold is True Value : {short_hold}")
        short_hold = False
        log.debug(f'Set short_hold to False new Value = {short_hold}')
        profit_short = old_price_short-current_price 
        log.info(f"Exiting Trade at price {current_price} * qty {qty} = {current_price*qty}")
        log.info(f"Booked a profit of {profit_long} * {qty} = {profit_long*qty}")
        order_value = qty*current_price
        
        trade_book.append({'date':date,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty})



    if per_change_short >= stop_loss_precentage and short_hold:
        log.warning(f"Stoploss on Short Position Triggered since per_change_short : {per_change_short} is greater than stop_loss_precentage : {stop_loss_precentage}. in absolute term. and short_hold is True : {short_hold} NOTE :: per_change_should be positive since it is a short trade")
        short_hold = False
        log.debug('Assigned new value to short_hold False : {False}')
        stop_loss_short = True
        log.debug(f'Since Stoploss triggered setting stop_loss_short to Truew new value : {stop_loss_short}')
        loss_on_short = old_price_long-current_price
        log.warning(f'Exiting short Trade at price : {current_price}. Booked a loss of {loss_on_short} *qty = {loss_on_short*qty}')
        
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit short Stoploss','price':current_price,'order_value':int(order_value),'profit':loss*qty})

    
    if (not short_hold) and (not stop_loss_short):
        
        log.info(f"Going Short since Both short_hold : {short_hold} and stop_loss_short : {stop_loss_short} are False. ")
        short_hold = True
        log.info(f'Shorting at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_short = current_price
        log.debug(f"Setting old_short_price to {current_price}. value after setting {old_price_short}")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'short','price':current_price,'order_value':int(order_value),'profit':0})

In [49]:
trade_book_df = pd.DataFrame(trade_book)
trade_book_df

,date,type,price,order_value,profit,charges,profit_after_charges
0,2024-03-13 09:15:00+05:30,long,47430.351562,1422910,0.000000,92.687316,-92.687316
1,2024-03-13 09:15:00+05:30,short,47430.351562,1422910,0.000000,405.727637,-405.727637
2,2024-03-13 09:16:00+05:30,exit short,47376.250000,1421287,1623.046875,92.638625,1530.408250
3,2024-03-13 09:16:00+05:30,short,47376.250000,1421287,0.000000,405.321875,-405.321875
4,2024-03-13 09:31:00+05:30,exit long,47325.449219,1419763,-3147.070312,404.940869,-3552.011182
5,2024-03-13 09:31:00+05:30,exit short,47325.449219,1419763,1524.023438,92.592904,1431.430533
6,2024-03-13 09:31:00+05:30,short,47325.449219,1419763,0.000000,404.940869,-404.940869
7,2024-03-13 09:54:00+05:30,exit short,47270.601562,1418118,1645.429688,92.543541,1552.886146
8,2024-03-13 09:54:00+05:30,short,47270.601562,1418118,0.000000,404.529512,-404.529512
9,2024-03-13 10:14:00+05:30,exit short,47430.750000,1422922,4804.453125,405.730625,4398.722500


In [50]:
sum(trade_book_df['profit'])

6449.8828125

In [51]:
sum(trade_book_df['profit_after_charges'])

3648.2290386718746